# **Transformers and XAI: practice**
**Добро пожаловать на завершающую практику курса, друзья!**

Прежде чем приступить, хочу сказать вам спасибо! Это мой первый общедоступный курс, и мне было важно сделать его интересным. Ваш фидбек, замечания и идеи делали и делают его лучше, поэтому feel free to делиться ими в чатах курса. Я благодарна вам за внимание и упорство в задачах. Вы — лучшие!

**Перейдем к уроку! :)**

Интерпретация моделей является интересной и захватывающей задачей и трансформеры с их механизмом внимания — не исключение. Однако, работая с ними, важность **качество предобучения**.

Как правило, если при анализе важности в модели опираться на карты внимания, нужно использовать *хорошо предобученную* модель для вашей задачи. Это будет гарантировать вам, что внимание модели хорошо заточено на ваши данные и, как следствие, из него действительно *можно пытаться* извлечь рабочие гипотезы о данных.  

Однако, не вниманием единым! Такие методы как SHAP, LIME, Integrated Gradients и другие тоже могут быть полезны.

**В этом уроке мы:**

- Научимся визуализировать механизм внимания (attention weights);
- Применим методы интерпретации (такие как SHAP) для анализа решений, принимаемых моделью;
- Познакомитесь с интрументарием, позволяющим визуализировать модели типа transformer в пару строк кода;
- Обсудим важность интерпретации моделей в реальных приложениях и рассмотрим практические кейсы.

**Будет красиво!** Приятного кодинга! :)

![](https://ucarecdn.com/1a50e7f8-b09b-4b85-b928-b50efe5ecbba/)

Начнем с установки всех нужных библиотек. Не пугайтесь, на это может потребоваться 2-5 минут.

In [ ]:
!pip install transformers shap transformers-interpret ferret-xai -q # Как всегда подтянем все нужные библиотеки

И импортируем необходимые сущности.

In [ ]:
# MODEL
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch

# XAI Libs
from transformers_interpret import SequenceClassificationExplainer
from ferret import Benchmark
import shap

# VIZ
import numpy as np
import matplotlib.pyplot as plt


Будем работать над классической и очень красивой задачей — задачей классификации тональности текста. Чтобы не тратить время, возьмём предобученную модель `distilbert`.

**DistilBERT** — это более маленькая и прозводительная модель, чем базовый BERT (Bidirectional Encoder Representations from Transformers). Она имеет на 40% меньше параметров, чем google-bert/bert-base-uncased и работает на 60% быстрее, сохраняя при этом более 95% производительности оригинальной модели.

In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884


Определим текст для работы. Вы можете задать также свой, но для выполнения заданий на степик, используйте оригинальный.


In [ ]:
data = ["You look terrible! Maybe you should rest?"]

In [ ]:
txt_tokens = tokenizer(data[0], return_tensors="pt", add_special_tokens=True) # Токенизируем слова
labels = tokenizer.tokenize(data[0], add_special_tokens=True) # Сохраним labels для построения будущей карты внимания

**Какой токен id соответствует первому слову?**
Ваш ответ здесь

Внимание будем строить на основе взятия медианы по выходам трансформера. По умолчанию количество карт внимания в модели есть **num_layers*num_heads**.

**Проверьте, сколько карт внимания у модели distillbert?**

In [ ]:
# Ваш ответ здесь

Поскольку инференс трансформера трудоёмкая задача, для получения карт внимания нужно указывать специальный гиперпараметр — `output_attentions=True`. Для визуализации используем медианное значение по всем головам.

**Note:** Никто не ограничивает вас от взятия среднего, минимума, максимума или другой статистики. Какая будет наиболее информативной — сложно сказать, однако по умолчанию в работах наиболее часто применяется медиана.

In [ ]:
predictions = model(**txt_tokens, output_attentions=True)

In [ ]:
all_attentions = np.array([i.detach().numpy() for i in predictions.attentions[:]]).squeeze(1)

med_layers_att = np.median(all_attentions, axis=0)
med_heads_att = np.median(med_layers_att, axis=0)


In [ ]:
# Визуализируем внимание

fig, ax = plt.subplots(figsize=(6, 12))
ax.imshow(np.median(all_attentions[3], axis=0))


ax.set_xticks(np.arange(len(labels)), labels=labels)
ax.set_yticks(np.arange(len(labels)), labels=labels);


Чтобы сгладить оценку, можно использовать ещё одно усреднение, но уже на данном этапе видно, что медианная карта не совсем информативна. Так происходит, потому что каждая голова может брать на себя разные паттерны входной последовательности. Кроме того, модель училась работать с тональностью отзывов, а не c тональностью в целом.

Но минусы не значат, что мы не можем продолжить исследование. К модели типа трансформер могут быть применены также другие методы.


Начнем с shap.

### **shap X transformers**

In [ ]:
classifier = pipeline('text-classification', top_k=None, model=model, tokenizer=tokenizer)
explainer = shap.Explainer(classifier)

print('Predictions')
print(classifier(data))

# Высчитаем значения shap
shap_values = explainer(data)

Predictions
[[{'label': 'NEGATIVE', 'score': 0.9995892643928528}, {'label': 'POSITIVE', 'score': 0.0004107290878891945}]]


  0%|          | 0/110 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:15, 15.39s/it]               


In [ ]:
shap.plots.text(shap_values)

**Проанализируйте график shap, как мы делали это раньше. Чему равен прогноз модели?**

**Чему равно base value класса NEGATIVE?**

Рассмотрим еще один тип графика — barplot. Можно визуализировать влияние токенов на "POSITIVE" и "NEGATIVE" классы отдельно.

In [ ]:
# Влияние токенов на Negative класс
shap.plots.bar(shap_values[0, :, "NEGATIVE"], max_display=10)

In [ ]:
# Влияние токенов на Positive класс
shap.plots.bar(shap_values[0, :, "POSITIVE"], max_display=10)

### **transformers-interpret X transformers**

Ещё одна механическая реализация способа заглянуть внутрь модели — библиотека`transformers-interpret`. Как вы можете заметить из названия — она разработа специально под модели типа трансформер.

Из плюсов:
- Transformers Interpret позволяет объяснить любую модель transformer всего в две строки
- Доступны объяснения как для текстовых моделей, так и для CV моделей компьютерного зрения
- Визуализации доступны ​​в виде легко сохраняемых файлов png и html

Из минусов:
- Обертка даёт меньше контроля над получением объяснения

В целом, данная библиотека отличный способ получить объяснение быстро. Она позволяет экономить время и, кроме того, облегчает создание отчета по объяснением за счет удобного сохранения файлов.

Для примеров с MultiLabel, Zero Shot Classification, Question Answering, Token Classification (NER) и Image Classification переходите [сюда](https://github.com/cdpierse/transformers-interpret/tree/master).

In [ ]:
cls_explainer = SequenceClassificationExplainer(model, tokenizer) # согласно философии библиотеки, объявляем классификатор
word_attributions = cls_explainer(data[0]) # получаем атрибуции в пару строк

In [ ]:
cls_explainer.visualize("distilbert_viz.html", true_class='NEGATIVE') # осуществляем визуализацию атрибуций

In [ ]:
fig, ax = plt.subplots()

y_pos = np.arange(len(word_attributions))
word_attributions_numbers = [i[1] for i in word_attributions]
word_attributions_labels = [i[0] for i in word_attributions]

ax.barh(y_pos, word_attributions_numbers)
ax.set_yticks(y_pos, labels=word_attributions_labels)
ax.set_title('Word attributions with transformers-interpret')
plt.show()

### **FERRET x transformers**

Как вы можете заметить, атрибуции, получаемые разными методами в рамках одной модели — различны. Мы видели это на протяжении курса и наблюдаем сейчас. Сказать, что лучше — нельзя. Но, быть может, всё только впереди =)


Из плюсов фреймворка:
- быстрый сравнительный анализ атрибуций разных методов объяснения
- поддержка аудиоданных и объяснений для них

Из минусов:
- Аналогично transformers-interpet, обертка даёт меньше контроля над получением объяснения

In [ ]:
bench = Benchmark(model, tokenizer)

explanations = bench.explain('You look terrible! Maybe you should rest?', target=0)
evaluations = bench.evaluate_explanations(explanations, target=0)

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

Explanation eval:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
bench.show_table(explanations)

**Какой токен имеет в ferret table наибольшую оценку во влиянии на прогноз?**

 На платформе вас ждут еще несколько библиотек для работы с трансформерами. 🏎

 Спасибо за работу, друзья! Жду вас в новых курсах! :)